In [1]:
import csv
import numpy as npmy
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import mysql.connector
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import seaborn as sb
%matplotlib inline

In [2]:
#Using facebook fasttext for classification.
import fasttext as ft

In [3]:
try:
    cnx = mysql.connector.connect(user='prat04',password='Dularu$9',host='127.0.0.1',database='stackoverflow_stackexchange')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err) 
cursor=cnx.cursor()

In [15]:
postDF=pd.read_csv('/home/cloakedprat/csv1/post.csv',names=['ViewCount','Body','Title','Tags'])

In [53]:
post_trim=pd.DataFrame(columns=['Body','Tags'])
post_trim.Body,post_trim.Tags=postDF.Body,postDF.Tags

In [55]:
postDF

,ViewCount,Body,Title,Tags
0,30258,<p>I want to use a track-bar to change a form'...,While applying opacity to a form should we use...,<c#><winforms><type-conversion><decimal><opacity>
1,14387,<p>I have an absolutely positioned <code>div</...,but not on Firefox or Safari.</p>&#xA;&#xA;<p...,it works. If the parent is relatively positioned
2,372893,<p>Given a <code>DateTime</code> representing ...,How do I calculate someone's age in C#?,<c#><.net><datetime>
3,117846,<p>Given a specific <code>DateTime</code> valu...,How can relative time be calculated in C#?,<c#><datetime><datediff><relative-time-span>
4,133301,<p>Is there any standard way for a Web Server ...,Determine a User's Timezone,<javascript><html><browser><timezone><timezone...
5,89146,<p>What is the difference between <a href=\htt...,Difference between Math.Floor() and Math.Trunc...,<.net><math>
6,72691,<p>How do you expose a LINQ query as an ASMX w...,Filling a DataSet or DataTable from a LINQ que...,<c#><linq><web-services><.net-3.5>
7,49844,<p>How do I store binary data in <a href=\http...,Binary Data in MySQL,<mysql><database><binary-data><data-storage>
8,35907,<p>Solutions are welcome in any language. :-) ...,What is the fastest way to get the value of π?,<performance><algorithm><language-agnostic><un...
9,44863,<p>If I have a <code>trigger</code> <code>befo...,Throw an error in a MySQL trigger,<mysql><database><triggers>


In [54]:
post_trim

,Body,Tags
0,<p>I want to use a track-bar to change a form'...,<c#><winforms><type-conversion><decimal><opacity>
1,<p>I have an absolutely positioned <code>div</...,it works. If the parent is relatively positioned
2,<p>Given a <code>DateTime</code> representing ...,<c#><.net><datetime>
3,<p>Given a specific <code>DateTime</code> valu...,<c#><datetime><datediff><relative-time-span>
4,<p>Is there any standard way for a Web Server ...,<javascript><html><browser><timezone><timezone...
5,<p>What is the difference between <a href=\htt...,<.net><math>
6,<p>How do you expose a LINQ query as an ASMX w...,<c#><linq><web-services><.net-3.5>
7,<p>How do I store binary data in <a href=\http...,<mysql><database><binary-data><data-storage>
8,<p>Solutions are welcome in any language. :-) ...,<performance><algorithm><language-agnostic><un...
9,<p>If I have a <code>trigger</code> <code>befo...,<mysql><database><triggers>


In [2]:
#Create DF of title and tags.
del postDF.Body

In [153]:
#Test_train split
x_train,x_test,y_train,y_test= sk.model_selection.train_test_split(post_tagPredict.Title+post_tagPredict.Body,post_tagPredict.Tags)

In [60]:
#Format data for one question per line with labels separated by 'label_prefix'. This can be any thing.
classifier=fasttext.supervised('/home/cloakedprat/csv1/tagPosts.csv','model',label_prefix='__label__')

In [ ]:
#Preprocess data to remove opening and closing braces,code gibberish, and special characters like #$%& etc.
#This helps increase the classifiation score.